In [1]:
import sys

# sys.path.append(r'E:\Projects\SA - R&D\multimodal_assistant\llm')

import fitz
import pandas as pd
import os
from langchain.docstore.document import Document
from llm_client import LLMClient
from PIL import Image
from io import BytesIO
import base64

/Users/SathyaPriya.Turaga/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/SathyaPriya.Turaga/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/SathyaPriya.Turaga/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
def get_b64_image(image_path):
    image = Image.open(image_path).convert("RGB")
    buffered = BytesIO()
    image.save(buffered, format="JPEG", quality=20)  # quality=20 is a workaround (WAR)
    b64_string = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return b64_string

def is_graph(image_path):
    # Placeholder function for graph detection logic
    # Implement graph detection algorithm here
    neva = LLMClient("neva_22b")
    b64_string = get_b64_image(image_path)
    res = neva.multimodal_invoke(b64_string, creativity = 0, quality = 9, complexity = 0, verbosity = 9).content
    # print(res)
    if "graph" in res or "plot" in res or "chart" in res:
        return True
    else:
        return False

def process_graph(image_path):
    # Placeholder function for graph processing logic
    # Implement graph processing algorithm here
    # Call DePlot through the API
    deplot = LLMClient("deplot")
    b64_string = get_b64_image(image_path)
    res = deplot.multimodal_invoke(b64_string)
    deplot_description = res.content
    print("\n\ndeplot_description :",deplot_description)
    mixtral = LLMClient(model_name="mixtral_8x7b")
    response = mixtral.chat_with_prompt(system_prompt="Your responsibility is to explain charts. You are an expert in describing the responses of linearized tables into plain English text for LLMs to use.", 
                             prompt="Explain the following linearized table. " + deplot_description)
    full_response = ""
    for chunk in response:
        full_response += chunk
    # print(full_response)
    print("full_response :",full_response)
    return full_response

def extract_text_around_item(text_blocks, bbox, page_height, threshold_percentage=0.1):
    before_text, after_text = "", ""
    vertical_threshold_distance = page_height * threshold_percentage
    horizontal_threshold_distance = bbox.width * threshold_percentage  # Assuming similar threshold for horizontal distance

    for block in text_blocks:
        block_bbox = fitz.Rect(block[:4])
        vertical_distance = min(abs(block_bbox.y1 - bbox.y0), abs(block_bbox.y0 - bbox.y1))
        horizontal_overlap = max(0, min(block_bbox.x1, bbox.x1) - max(block_bbox.x0, bbox.x0))

        # Check if within vertical threshold distance and has horizontal overlap or closeness
        if vertical_distance <= vertical_threshold_distance and horizontal_overlap >= -horizontal_threshold_distance:
            if block_bbox.y1 < bbox.y0 and not before_text:
                before_text = block[4]
            elif block_bbox.y0 > bbox.y1 and not after_text:
                after_text = block[4]
                break

    return before_text, after_text

def process_text_blocks(text_blocks):
    char_count_threshold = 500  # Threshold for the number of characters in a group
    current_group = []
    grouped_blocks = []
    current_char_count = 0

    for block in text_blocks:
        if block[-1] == 0:  # Check if the block is of text type
            block_text = block[4]
            block_char_count = len(block_text)

            if current_char_count + block_char_count <= char_count_threshold:
                current_group.append(block)
                current_char_count += block_char_count
            else:
                if current_group:
                    grouped_content = "\n".join([b[4] for b in current_group])
                    grouped_blocks.append((current_group[0], grouped_content))
                current_group = [block]
                current_char_count = block_char_count

    # Append the last group
    if current_group:
        grouped_content = "\n".join([b[4] for b in current_group])
        grouped_blocks.append((current_group[0], grouped_content))

    return grouped_blocks

In [5]:
from IPython.display import display, HTML

def parse_all_tables(filename, page, pagenum, text_blocks, ongoing_tables):
    table_docs = []
    table_bboxes = []
    ctr = 1
    try:
        tables = page.find_tables(horizontal_strategy = "lines_strict", vertical_strategy = "lines_strict")
    except Exception as e:
        print(f"Error during table extraction: {e}")
        
        return table_docs, table_bboxes, ongoing_tables
    if tables:
        for tab in tables:
            # display(tab['dataframe'])
            if tab.header.external:
                # Check if this table is a continuation of a table from a previous page
                previous_table = ongoing_tables.get(pagenum - 1, None)
                if previous_table:
                    # Merge the current table with the previous part
                    combined_df = pd.concat([previous_table['dataframe'], tab.to_pandas()])
                    ongoing_tables[pagenum] = {"dataframe": combined_df, "bbox": bbox}
                continue
            if not tab.header.external:
                pandas_df = tab.to_pandas()
                # pandas_df = pandas_df.drop(pandas_df.columns[1], axis=1, inplace=True)
                print("Table as DF\n")
                display(pandas_df)
                # exit()
                tablerefdir = os.path.join(os.getcwd(), "vectorstore/table_references")
                if not os.path.exists(tablerefdir):
                    os.makedirs(tablerefdir)
                df_xlsx_path = os.path.join(tablerefdir, f"table{ctr}-page{pagenum}.xlsx")
                pandas_df.to_excel(df_xlsx_path,index=False)
                bbox = fitz.Rect(tab.bbox)
                table_bboxes.append(bbox)

                # Find text around the table
                before_text, after_text = extract_text_around_item(text_blocks, bbox, page.rect.height)

                table_img = page.get_pixmap(clip=bbox)
                table_img_path = os.path.join(tablerefdir, f"table{ctr}-page{pagenum}.jpg")
                
                pil_image = Image.frombytes("RGB", [table_img.width, table_img.height], table_img.samples)

                dpi_factor = 100 
                pil_image.save(table_img_path, dpi=(300 * dpi_factor, 300 * dpi_factor))
                # description = process_graph(table_img_path)
                ctr += 1

                caption = before_text.replace("\n", " ") + after_text.replace("\n", " ")
                if before_text == "" and after_text == "":
                    caption = " ".join(tab.header.names)


                table_metadata = {
                    "source": f"{filename[:-4]}-page{pagenum}-table{ctr}",
                    "dataframe": df_xlsx_path,
                    "image": table_img_path,
                    "caption": caption,
                    "type": "table",
                    "page_num": pagenum
                }
                all_cols = ", ".join(list(pandas_df.columns.values))
                
                doc = Document(page_content="This is a table with the caption: " + caption + f"\nThe columns are {all_cols}", metadata=table_metadata)
                table_docs.append(doc)
                
                # print(table_docs)
    return table_docs, table_bboxes, ongoing_tables

In [4]:
def parse_all_images(filename, page, pagenum, text_blocks):
    image_docs = []
    image_info_list = page.get_image_info(xrefs=True)
    page_rect = page.rect  # Get the dimensions of the page

    for image_info in image_info_list:
        xref = image_info['xref']
        if xref == 0:
            continue  # Skip inline images or undetectable images

        img_bbox = fitz.Rect(image_info['bbox'])
        # Check if the image size is at least 5% of the page size in any dimension
        if img_bbox.width < page_rect.width / 20 or img_bbox.height < page_rect.height / 20:
            continue  # Skip very small images

        # Extract and save the image
        extracted_image = page.parent.extract_image(xref)
        image_data = extracted_image["image"]
        imgrefpath = os.path.join(os.getcwd(), "vectorstore/image_references")
        if not os.path.exists(imgrefpath):
            os.makedirs(imgrefpath)
        image_path = os.path.join(imgrefpath, f"image{xref}-page{pagenum}.png")
        with open(image_path, "wb") as img_file:
            img_file.write(image_data)

        # Find text around the image
        before_text, after_text = extract_text_around_item(text_blocks, img_bbox, page.rect.height)
        # skip images without a caption, they are likely just some logo or graphics
        if before_text == "" and after_text == "":
            continue

        # Process the image if it's a graph
        image_description = " "
        if is_graph(image_path):
            image_description = process_graph(image_path)

        # Combine the texts to form a caption
        caption = before_text.replace("\n", " ") + image_description + after_text.replace("\n", " ")

        image_metadata = {
            "source": f"{filename[:-4]}-page{pagenum}-image{xref}",
            "image": image_path,
            "caption": caption,
            "type": "image",
            "page_num": pagenum
        }
        image_docs.append(Document(page_content="This is an image with the caption: " + caption, metadata=image_metadata))
    return image_docs

In [6]:
def get_pdf_documents(filepath):
    all_pdf_documents = []
    ongoing_tables = {}
    try:
        f = fitz.open(filepath)
    except Exception as e:
        print(f"Error opening or processing the PDF file: {e}")
        return []

    for i in range(len(f)):
        page = f[i]
        page_docs = []

        # Process text blocks
        initial_text_blocks = page.get_text("blocks", sort=True)

        # Define thresholds for header and footer (10% of the page height)
        page_height = page.rect.height
        header_threshold = page_height * 0.1
        footer_threshold = page_height * 0.9

        # Filter out text blocks that are likely headers or footers
        text_blocks = [block for block in initial_text_blocks if block[-1] == 0 and not (block[1] < header_threshold or block[3] > footer_threshold)]

        # Group text blocks by character count
        grouped_text_blocks = process_text_blocks(text_blocks)

        # Extract tables and their bounding boxes
        table_docs, table_bboxes, ongoing_tables = parse_all_tables(filepath, page, i, text_blocks, ongoing_tables)
        page_docs.extend(table_docs)

        # Extract and process images
        # image_docs = parse_all_images(filepath, page, i, text_blocks)
        # page_docs.extend(image_docs)

        # Process grouped text blocks
        text_block_ctr = 0
        for heading_block, content in grouped_text_blocks:
            text_block_ctr +=1 
            heading_bbox = fitz.Rect(heading_block[:4])
            # Check if the heading or its content overlaps with table or image bounding boxes
            if not any(heading_bbox.intersects(table_bbox) for table_bbox in table_bboxes):
                bbox = {"x1": heading_block[0], "y1": heading_block[1], "x2": heading_block[2], "x3": heading_block[3]}
                text_doc = Document(page_content=f"{heading_block[4]}\n{content}", metadata={**bbox, "type": "text", "page_num": i, "source": f"{filepath[:-4]}-page{i}-block{text_block_ctr}"})
                page_docs.append(text_doc)

        all_pdf_documents.append(page_docs)

    f.close()
    return all_pdf_documents

In [9]:
from difflib import SequenceMatcher

def remove_duplicate_paragraphs(text):
    # Split the text into paragraphs
    paragraphs = text.split("\n\n")
    
    # Remove duplicates while preserving the order
    unique_paragraphs = []
    seen_paragraphs = set()
    for i in range(len(paragraphs) - 1):
        # Compare current paragraph with the next one
        similarity = SequenceMatcher(None, paragraphs[i], paragraphs[i+1]).ratio()
        
        # If similarity is less than 0.9 (adjust as needed), consider them as different
        if similarity < 0.9:
            if paragraphs[i] not in seen_paragraphs:
                unique_paragraphs.append(paragraphs[i])
                seen_paragraphs.add(paragraphs[i])
    
    # Add the last paragraph
    if paragraphs[-1] not in seen_paragraphs:
        unique_paragraphs.append(paragraphs[-1])
    
    # Join the unique paragraphs back into a single string
    result_text = "\n\n".join(unique_paragraphs)
    
    return result_text

In [10]:
def main(pdf_filepath,nv_api_key='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'):
    

    # Set the value of NVIDIA_API_KEY environment variable
    os.environ['NVIDIA_API_KEY'] = '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'
    # Call the function to extract documents from the PDF
    pdf_documents_dl = get_pdf_documents(pdf_filepath)
    # Initialize an empty string to store the concatenated documents
    result_string = ""

    # Iterate through the extracted documents
    for page_documents in pdf_documents_dl:
        for document in page_documents:
            # Concatenate the page content to the result string
            result_string += document.page_content + "\n\n"

    result = remove_duplicate_paragraphs(result_string)
    
    return result

In [11]:
from time import time

st = time()
result = main(r"/Users/SathyaPriya.Turaga/Downloads/companycar.pdf")
ed = time()

tot = ed-st

print("Tot time :",tot)

Table as DF



Commute Type  \
0                                      National role   
1                                 Regional/Area role   
2                                 Regional/Area role   
3                                        Single Site   
4  Dual or Multi site role or\nroles with two or ...   
5                                Temporary Workplace   
6                    Secondment/Temporary\nWorkplace   
7                                              Other   
8                                              Other   

                                         Description  \
0  You travel the country\nand do not have a\ndef...   
1  Living OFF region\ne.g., regional support\ntea...   
2  Living ON region\ne.g., Regional support\nteam...   
3   Office, DC or Branch\nbased.\ne.g., CDM, RDM, AM   
4  You are responsible for\ntwo or more locations...   
5  You are working at the\nlocation for a period ...   
6  You are on secondment\nto a location for a per...   
7  I am dual sited (work\nregularly in two\nlocat...   
8  I am dual sited (work\nregularly in two sites)...   

  What is your normal place of work and what\ncan you claim for  
0  You can claim for all journeys as they are not...             
1  You cannot claim for travel to the nearest\nbr...             
2  You cannot claim for travel to your normal bas...             
3  Cannot claim mileage as this is seen as your\n...             
4  You cannot claim for your nominated locations\...             
5  If you are covering someone's role or working\...             
6  If you are on secondment, you can claim your\n...             
7  Being cost conscious we do not expect you to\n...             
8  You would have to take off normal daily travel...

Table as DF



Other  \
0                                              Other   
1  * A normal place of work or ordinary commute i...   
2  * Any breach of policy could lead to the colle...   

  I rotate my meetings\nwithin a small proximity\non a weekly basis  \
0  My meetings are only for\na couple of hours wi...                  
1                                               None                  
2                                               None                  

  If your meetings are in sites within 10 square\nmiles of each other, they would be classed as an\nordinary commute and cannot be claimed for.  
0  This would be classed as your normal daily\nco...                                                                                             
1                                               None                                                                                             
2                                               None

Tot time : 0.49420595169067383


In [13]:
print (result);

Ordinary Commute Policy 

February 2023 

 
 
 
 
 
 
 
 
 
 

What is ordinary commute? .......................................................................................................................... 3 

1. 
What is classed as a permanent workplace? ......................................................................................... 3 

2. 
Roles with one normal workplace .......................................................................................................... 3 

3. 
Roles with two or more normal workplaces, Dual located or multi-sited roles ........................................ 3 

4. 
Regionally based roles .......................................................................................................................... 4 

b. 
You live outside your region ............................................................................................................... 4 

5. 
Nationally based roles ..........................................